<a href="https://colab.research.google.com/github/S2-ai-team/BBL-AI/blob/main/BBL_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from flask import Flask, request, jsonify
import os

# Colab 환경에 맞춘 경로 설정
MODEL_PATH = '/content/rgbcmyw_emotion_model.h5'
TOKENIZER_PATH = '/content/rgbcmyw_tokenizer.json'

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

class Config:
    MAX_LEN = 32
    MODEL_SAVE_PATH = MODEL_PATH
    TOKENIZER_SAVE_PATH = TOKENIZER_PATH
    VOCAB_SIZE = 10000


def create_model(vocab_size, num_labels):
    input_layer = tf.keras.layers.Input(shape=(Config.MAX_LEN,), dtype='float32')
    embedding_layer = tf.keras.layers.Embedding(vocab_size, 128)(input_layer)
    dropout_layer = tf.keras.layers.Dropout(0.3)(embedding_layer)
    conv1 = tf.keras.layers.Conv1D(128, 3, activation='relu')(dropout_layer)
    pool1 = tf.keras.layers.GlobalMaxPooling1D()(conv1)
    conv2 = tf.keras.layers.Conv1D(128, 4, activation='relu')(dropout_layer)
    pool2 = tf.keras.layers.GlobalMaxPooling1D()(conv2)
    conv3 = tf.keras.layers.Conv1D(128, 5, activation='relu')(dropout_layer)
    pool3 = tf.keras.layers.GlobalMaxPooling1D()(conv3)
    concat = tf.keras.layers.Concatenate()([pool1, pool2, pool3])
    dense = tf.keras.layers.Dense(128, activation='relu')(concat)
    final_dropout = tf.keras.layers.Dropout(0.5)(dense)
    output_layer = tf.keras.layers.Dense(num_labels, activation='softmax')(final_dropout)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


def load_prediction_assets():
    if not os.path.exists(Config.TOKENIZER_SAVE_PATH):
        print("🔥 토크나이저 없음")
        return None, None, None
    try:
        with open(Config.TOKENIZER_SAVE_PATH, 'r', encoding='utf-8') as f:
            tokenizer_json = json.load(f)
            tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(tokenizer_json)
    except Exception as e:
        print(f"🔥 토크나이저 로드 실패: {e}")
        return None, None, None

    emotion_map = {
        '분노': 'r', '슬픔': 'b', '기쁨': 'y', '행복': 'r',
        '공포': 'c', '혐오': 'm', '놀람': 'w', '중립': 'g'
    }
    all_labels = sorted(list(set(emotion_map.values())))
    le = LabelEncoder()
    le.fit(all_labels)

    vocab_size = Config.VOCAB_SIZE
    num_labels = len(le.classes_)
    model = create_model(vocab_size, num_labels)

    if not os.path.exists(Config.MODEL_SAVE_PATH):
        print("🔥 모델 파일 없음")
        return None, None, None
    try:
        model.load_weights(Config.MODEL_SAVE_PATH)
    except Exception as e:
        print(f"🔥 모델 가중치 로드 실패: {e}")
        return None, None, None

    return model, tokenizer, le


def predict_emotion(text, model, tokenizer, le):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=Config.MAX_LEN)
    prediction = model.predict(padded_seq, verbose=0)
    label_index = np.argmax(prediction)
    confidence = np.max(prediction) * 100
    predicted_label = le.inverse_transform([label_index])[0]
    if predicted_label == 'y' and confidence < 50:
        predicted_label = 'r'
    return predicted_label, f"{confidence:.2f}%"


app = Flask(__name__)
model, tokenizer, le = load_prediction_assets()

@app.route("/", methods=["GET"])
def home():
    return "✅ 감정 예측 API 작동 중!"

@app.route("/predict", methods=["POST"])
def predict():
    if not model or not tokenizer or not le:
        return jsonify({"error": "모델 또는 토크나이저 로드 실패"}), 500
    try:
        data = request.json
        text = data.get("text", "")
        if not text:
            return jsonify({"error": "text 파라미터가 필요합니다."}), 400
        label, confidence = predict_emotion(text, model, tokenizer, le)
        return jsonify({"label": label, "confidence": confidence})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=7860)


🔥 토크나이저 없음
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7860
 * Running on http://172.28.0.12:7860
INFO:werkzeug:Press CTRL+C to quit
